In [1]:
import pyedflib
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
from scipy.signal import butter, filtfilt, spectrogram
import os
import mne

In [2]:
def debauchy_filter(data, lowcut, highcut, fs, order=6):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='bandpass')
    filtered_data = filtfilt(b, a, data)
    return filtered_data

save_dir = "images/chb02_seizure/16"

os.makedirs(save_dir, exist_ok=True)

index_file = "image_index.txt"
if os.path.exists(index_file):
    with open(index_file, 'r') as f:
        image_index = int(f.read())
else:
    image_index = 0

image_index += 1

with open(index_file, 'w') as f:
    f.write(str(image_index))

lowcut = 1.0  
highcut = 50.0  
fs = 256.0

start_time = 120
end_time = 170

start_sample = int(start_time * 256)
end_sample = int(end_time * 256)

start_sample_50s = max(0, start_sample - 50 * 256)
end_sample_50s = start_sample_50s + 50 * 256

f = pyedflib.EdfReader("data/chb02/chb02_16.edf")

for signal_index in range(23):
    signal_data = f.readSignal(signal_index)
    
    signal_data_50s = signal_data[start_sample_50s:end_sample_50s]

    filtered_data_50s = debauchy_filter(signal_data_50s, lowcut, highcut, 256)

    frequencies, times, Sxx = spectrogram(filtered_data_50s, fs=256, noverlap=1)

    plt.figure(figsize=(10, 5))
    plt.pcolormesh(times, frequencies, 10 * np.log10(Sxx), shading='gouraud')
    plt.gca().set_axis_off()
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
    plt.margins(0, 0)
    plt.gca().xaxis.set_major_locator(matplotlib.ticker.NullLocator())
    plt.gca().yaxis.set_major_locator(matplotlib.ticker.NullLocator())
    
    image_name = f"image_{image_index}_{signal_index}.png"
    image_path = os.path.join(save_dir, image_name)

    plt.savefig(image_path, bbox_inches='tight', pad_inches=0)
    plt.close()

with open(index_file, 'w') as f:
    f.write(str(image_index))
